# LAB 2. Part 1. Data Cleaning

The input data can be messy with all sorts of issues like 
* missing or incorrect values (e.g. entries containing nothing, zero, nan or wrong quantity);
* misformatted records and entries (e.g. entries as strings instead of numeric or date-time or wrong number of entries per row);
* duplicate records and other.

In short a rule of thumb is that nothing can be taken for granted and need to be verified with format, value and sanity checks. Some issue just won't let you upload the data.
With others Python could let you through but as we'll see in the case in the second notebook, if ingored, those issues can be substaintial enough to completely derail the analysis.

So data cleaning is an important first step in any data analyics project. Consider an example of some real-estate data.

In [1]:
import pandas as pd
import numpy as np
import os
import urllib

In [2]:
#read the data and visualize as text to see what's inside

In [7]:
if not os.path.exists('Data'):
    !mkdir Data

fname = 'RE_example11.csv'
f = open(fname, 'r')
print(f.read())
f.close()

SALE DATE,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
3/31/15, 8647 15TH AVENUE,11228,1547,1428,1930,758000
3/31/15, 8647 15TH AVENUE,11228,1547,1428,1930,758000
3/31/15, 8647 15TH AVENUE,11228,1547,1428,1930,758000

6/15/15, 55 BAY 10TH   STREET,11228,1933,1660,0,778000

9/16/15, 8620 19TH   AVENUE  ,11214,2417,2106,1930,0

5/29/15, 1906 86TH   STREET  ,11214,1900,2090,1931,1,365000

12/17/15, 50 BAY 23RD STREET  ,11214,2417,1672,1930,750000

May,6,2015, 1964 86TH   STREET  ,11214,1725,2112,1925,1,470000

4/30/15, 1970 86TH   STREET  ,11214,1725,2112,1931,1,790000

7/24/15, 1638 BENSON AVENUE  ,11214,3625,1825,1925,820,000

6/25/15, 1653 BATH AVENUE,11214,1260,1440,1930,0

6/25/15, 1651 BATH AVENUE,11214,1260,1440,1930,0

10/17/15, 1731 BATH AVENUE,11214,1112,1520,1920,0

5/12/15, 121 BAY 17TH   STREET,11214,1619,912,1910,0

11/2/15, 244 BAY 8TH STREET  ,11228,5720,5152,1930,0

12/18/15, 239 BAY8TH STREET   ,11228,5810,1575,1950,1,850000

12/7/15, 8804 17

We may see:

* some rows duplicated, 
* some having too many commas (perhaps due to using them to separate groups of digits in addition to separating fields),
* missing values,
* zero values
* none/nan values
* inconsistent date formats

This is of course a toy example where all those issues were deliberately "concetrated" within a small sample for illustration purposes, but you can expect similar issues with real-world data as well.

In [8]:
#pd.read_csv('data/RE_example11.csv') #if we try to read the data it will through an error due to format inconsistency between rows

In [9]:
re_sales = pd.read_csv('RE_example11.csv', error_bad_lines=False) #so use error_bad_lines flag to instruct pandas to skip the misformatted lines

b'Skipping line 10: expected 7 fields, saw 8\nSkipping line 14: expected 7 fields, saw 10\nSkipping line 16: expected 7 fields, saw 8\nSkipping line 18: expected 7 fields, saw 8\nSkipping line 30: expected 7 fields, saw 8\nSkipping line 32: expected 7 fields, saw 9\nSkipping line 40: expected 7 fields, saw 8\nSkipping line 70: expected 7 fields, saw 8\nSkipping line 76: expected 7 fields, saw 8\nSkipping line 91: expected 7 fields, saw 8\nSkipping line 96: expected 7 fields, saw 8\n'


In [11]:
re_sales #the data is read succesfully, while losing some lines; but some issue are still there

,SALE DATE,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
0,3/31/15,8647 15TH AVENUE,11228,1547,1428,1930,758000.0
1,3/31/15,8647 15TH AVENUE,11228,1547,1428,1930,758000.0
2,3/31/15,8647 15TH AVENUE,11228,1547,1428,1930,758000.0
3,6/15/15,55 BAY 10TH STREET,11228,1933,1660,0,778000.0
4,9/16/15,8620 19TH AVENUE,11214,2417,2106,1930,0.0
5,12/17/15,50 BAY 23RD STREET,11214,2417,1672,1930,750000.0
6,6/25/15,1653 BATH AVENUE,11214,1260,1440,1930,0.0
7,6/25/15,1651 BATH AVENUE,11214,1260,1440,1930,0.0
8,10/17/15,1731 BATH AVENUE,11214,1112,1520,1920,0.0
9,5/12/15,121 BAY 17TH STREET,11214,1619,912,1910,0.0


In [12]:
re_sales.describe() #use basic descriptive analysis to spot them

,ZIP CODE,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
count,42.000000,42.000000,42.000000,40.000000
mean,11220.333333,1697.190476,1888.261905,427950.250000
std,7.052648,800.496835,298.712690,396534.647534
min,11214.000000,912.000000,0.000000,0.000000
25%,11214.000000,1161.000000,1930.000000,0.000000
50%,11214.000000,1466.000000,1930.000000,662500.000000
75%,11228.000000,2116.500000,1943.750000,758000.000000
max,11228.000000,5152.000000,1960.000000,970000.000000


In [8]:
#we see LAND SQUARE FEET is not included, 
#meaining that it is perhaps treated as a string field, rather than date

In [13]:
re_sales['LAND SQUARE FEET'] = pd.to_numeric(re_sales['LAND SQUARE FEET'], errors='coerce') #convert to numeric, turning invalid parsing to NaN

In [14]:
(re_sales['SALE DATE'].min(), re_sales['SALE DATE'].max()) #also if we try getting a range for SALE DATE it does not work properly giving us text data

('1/13/15', 'Apr 24 2015')

In [15]:
re_sales['SALE DATE'] = pd.to_datetime(re_sales['SALE DATE']) #convert to data-time; it unifies variety of formats

In [16]:
#now descriptive analysis works as intended
(re_sales['SALE DATE'].min(), re_sales['SALE DATE'].max())

(Timestamp('2015-01-13 00:00:00'), Timestamp('2015-12-29 00:00:00'))

In [17]:
re_sales.describe() #however min values are 0 for the fiels that should not have zeros

,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
count,42.000000,41.000000,42.000000,42.000000,40.000000
mean,11220.333333,1988.073171,1697.190476,1888.261905,427950.250000
std,7.052648,768.055837,800.496835,298.712690,396534.647534
min,11214.000000,1112.000000,912.000000,0.000000,0.000000
25%,11214.000000,1551.000000,1161.000000,1930.000000,0.000000
50%,11214.000000,1844.000000,1466.000000,1930.000000,662500.000000
75%,11228.000000,2320.000000,2116.500000,1943.750000,758000.000000
max,11228.000000,5720.000000,5152.000000,1960.000000,970000.000000


In [18]:
#introduce basic sanity filtering, excluduing zero values
sanityindex = (re_sales['SALE PRICE'] > 0) & (re_sales['YEAR BUILT'] > 0)
re_sales = re_sales.loc[sanityindex]

In [19]:
re_sales = re_sales.loc[sanityindex]
re_sales.describe() #still min value for the sale price remains unrealistic

,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
count,23.000000,22.000000,23.000000,23.000000,23.000000
mean,11221.304348,1967.818182,1600.391304,1937.391304,710435.217391
std,7.150555,534.887406,607.553719,11.761799,237812.325123
min,11214.000000,1198.000000,992.000000,1910.000000,10.000000
25%,11214.000000,1603.250000,1162.000000,1930.000000,732500.000000
50%,11228.000000,1844.000000,1428.000000,1930.000000,750000.000000
75%,11228.000000,2275.500000,1950.000000,1950.000000,812000.000000
max,11228.000000,3867.000000,3160.000000,1960.000000,970000.000000


In [20]:
sanityindex = (re_sales['SALE PRICE'] > 5000) #remove properties worth less than 5000 (in the following case we'll see how one can get some insights on this kind of filtering)
re_sales = re_sales.loc[sanityindex]

In [21]:
re_sales 

,SALE DATE,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
0,2015-03-31,8647 15TH AVENUE,11228,1547.0,1428,1930,758000.0
1,2015-03-31,8647 15TH AVENUE,11228,1547.0,1428,1930,758000.0
2,2015-03-31,8647 15TH AVENUE,11228,1547.0,1428,1930,758000.0
5,2015-12-17,50 BAY 23RD STREET,11214,2417.0,1672,1930,750000.0
11,2015-08-24,1942 BATH AVENUE,11214,2013.0,1980,1910,895000.0
12,2015-09-14,2018 BATH AVENUE,11214,1198.0,1596,1925,745000.0
14,2015-10-08,310 BAY 10TH STREET,11228,3867.0,1399,1935,855000.0
15,2015-01-16,8846 16TH AVE,11214,1937.0,1240,1950,720000.0
16,2015-11-02,8870 16TH AVENUE,11214,1760.0,992,1950,620000.0
17,2015-10-21,1592 INDEPENDENCE AVENUE,11228,1844.0,992,1950,750000.0


In [22]:
#two remaining issues are nan values and duplicate rows address those below

In [23]:
re_sales = re_sales.loc[sanityindex].dropna()

In [24]:
re_sales = pd.DataFrame.drop_duplicates(re_sales) 
#new pandas versions have .drop_duplicates as a method of a dataframe enabling re_sales.drop_duplicated(inplace = True) 

In [25]:
re_sales #finally we can see that the index now have gaps as we dropped quite a few records

,SALE DATE,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
0,2015-03-31,8647 15TH AVENUE,11228,1547.0,1428,1930,758000.0
5,2015-12-17,50 BAY 23RD STREET,11214,2417.0,1672,1930,750000.0
11,2015-08-24,1942 BATH AVENUE,11214,2013.0,1980,1910,895000.0
12,2015-09-14,2018 BATH AVENUE,11214,1198.0,1596,1925,745000.0
14,2015-10-08,310 BAY 10TH STREET,11228,3867.0,1399,1935,855000.0
15,2015-01-16,8846 16TH AVE,11214,1937.0,1240,1950,720000.0
16,2015-11-02,8870 16TH AVENUE,11214,1760.0,992,1950,620000.0
17,2015-10-21,1592 INDEPENDENCE AVENUE,11228,1844.0,992,1950,750000.0
19,2015-11-20,277 BAY 14 STREET,11214,1795.0,1204,1930,762000.0
21,2015-04-24,308 BAY 14TH STREET,11214,1511.0,992,1950,650000.0


In [26]:
re_sales.reset_index(inplace = True, drop = True) #if we want it consistent we can reset index
re_sales

,SALE DATE,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
0,2015-03-31,8647 15TH AVENUE,11228,1547.0,1428,1930,758000.0
1,2015-12-17,50 BAY 23RD STREET,11214,2417.0,1672,1930,750000.0
2,2015-08-24,1942 BATH AVENUE,11214,2013.0,1980,1910,895000.0
3,2015-09-14,2018 BATH AVENUE,11214,1198.0,1596,1925,745000.0
4,2015-10-08,310 BAY 10TH STREET,11228,3867.0,1399,1935,855000.0
5,2015-01-16,8846 16TH AVE,11214,1937.0,1240,1950,720000.0
6,2015-11-02,8870 16TH AVENUE,11214,1760.0,992,1950,620000.0
7,2015-10-21,1592 INDEPENDENCE AVENUE,11228,1844.0,992,1950,750000.0
8,2015-11-20,277 BAY 14 STREET,11214,1795.0,1204,1930,762000.0
9,2015-04-24,308 BAY 14TH STREET,11214,1511.0,992,1950,650000.0


We can see that only 16 out of 41 records survived the data cleaning. It is not always that bad, but as we'll see from the following real world case if can sometimes be even worse...

## Homework - part 1

In [8]:
# please note that this is just a toy dataset, you don't need to follow the exactly the same data cleaning steps 
# when working on a real citibike dataset
url = 'citibike.csv'
citibike = pd.read_csv(url)

In [9]:
citibike.head()

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2020-08-01 00:00:00.8790,2020-08-01 00:33:34.7260,3295,Central Park W & W 96 St,40.791270,-73.964839,3992,W 169 St & Fort Washington Ave,40.842842,-73.942125,19706,Customer,1998,1.0
1,2020-08-01 00:00:01.5910,2020-08-01 01:56:44.0720,3144,E 81 St & Park Ave,40.776777,-73.959010,3671,E 81 St & 2 Ave,40.774779,-73.954275,37191,Subscriber,1966,1.0
2,2020-08-01 00:00:03.5810,2020-08-01 00:02:38.6340,387,Centre St & Chambers St,40.712733,-74.004607,387,Centre St & Chambers St,40.712733,-74.004607,42561,Customer,1969,0.0
3,2020-08-01 00:00:05.0130,2020-08-01 00:45:25.3270,3080,S 4 St & Rodney St,40.709340,-73.956080,3364,Carroll St & 5 Ave,40.675162,-73.981483,41607,Subscriber,1990,1.0
4,2020-08-01 00:00:05.0130,2020-08-01 00:45:25.3270,3080,S 4 St & Rodney St,40.709340,-73.956080,3364,Carroll St & 5 Ave,40.675162,-73.981483,41607,Subscriber,1990,1.0


In [10]:
citibike.shape

(100, 14)

### task 1
Filter out trips with unreasonal trip duration. Trip duration has to be a positive number, and shorter than 3 hours.

Hint: 
    1. convert starttime, endtime to timestamp at first
    2. use .astype('timedelta64[m]') to get trip duration in minutes

In [ ]:
# your code here

In [63]:
citibike.shape

(98, 15)

### task 2
Remove trip records which include unrealistic costumer age.

Hint:
    1. Customer age has to be less than 100.

In [ ]:
# your code here

In [65]:
citibike.shape

(95, 15)

### task 3
drop duplicated records

In [ ]:
# your code here

In [67]:
citibike.shape

(93, 15)

### task 4
Find records where start location or end location is outside of New York City, then delete them.

NYC latitude, longitude range is available at https://www1.nyc.gov/assets/planning/download/pdf/data-maps/open-data/nybb_metadata.pdf?ver=18c as
    
    West -74.257159 East -73.699215
    North 40.915568 South 40.495992

In [ ]:
# your code here

In [83]:
citibike.shape

(91, 15)